In [4]:
import json
import networkx as nx
import matplotlib.pyplot as plt

In [5]:
# Read data and draw some figures
import json

# Open and read the JSON file
with open('APF-south.json', 'r') as file:
    data = json.load(file)

# Print the data sample
print(data[:5]) 

[{'Recipient': 'B', 'Donor': ['AB', 'A']}, {'Recipient': 'O', 'Donor': ['AB', 'B', 'A']}, {'Recipient': 'O', 'Donor': ['AB', 'B', 'A']}, {'Recipient': 'B', 'Donor': ['AB', 'A']}, {'Recipient': 'B', 'Donor': ['A']}]


In [7]:
#import networkx as nx


# Create a graph
G = nx.DiGraph()

# Create nodes
counter = 0
for element in data: #for each element in the data it will create a node and add it to the counter
    #print(element)
    G.add_nodes_from([(counter, element)])
    counter += 1
#this gave us 720 nodes to check
print(G.nodes)

donor_recipient_compatibility = {'O': ['O', 'A', 'B', 'AB'], 'A': ['A', 'AB'], 'B':['B', 'AB'], 'AB':['AB']}
#which blood types are compatible with each other type o can do go to any but O must receive O

# Create edges (relations)
for node in G.nodes: #for all nodes
    donors = G.nodes[node]['Donor'] #pulls donor data from list 
    #print(donors)
    for donor in donors: #for each blood type in donor
        for vertex in G.nodes: #checks each node
            if node == vertex: continue #this skips if the current node is the same as the donor check
            recipient = G.nodes[vertex]['Recipient'] #pulls recipient data
            
            if recipient not in donor_recipient_compatibility[donor]: continue #this skips if blood type not donatable
            if recipient in donor_recipient_compatibility[donor]: G.add_edge(node, vertex)    #this adds an edge if 

print(G.number_of_edges())   
#print('edges:', list(G.edges))  #data checkpoints



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [ ]:
# Find cycles of length 3
import csv 

potential_cycle_3 = [] #init list of cycles


with open("cycles.csv","w",newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["Cycle Type", "Nodes"])
    writer.writerow(["Cycle 3"])

    H = G.to_undirected() #takes direction out of nodes to find cycles easier

    for (u,v) in H.edges: #adds all the close neighbors as potential cycles to check
        for k in nx.common_neighbors(H, u, v):
            potential_cycle_3.append((u,v,k))
                    
            
    cycle_3 = [] #init list of confirmed cycle 3

    for (u,v,k) in potential_cycle_3: #this iterates through each element in potential cycle to check if it is directed in G 
        if (u,v) in G.edges and (v,k) in G.edges and (k,u) in G.edges: #only appends if cycle completes
            cycle_3.append((u,v,k,u)) 
            writer.writerow(["Cycle of 3", [u,v,k,u]]) 

        if (v,u) in G.edges and (u,k) in G.edges and (k,v) in G.edges: #checks the reverse of the node since it is undirected
            cycle_3.append((v,u,k,v))
            writer.writerow(["Cycle of 3", [v,u,k,v]])
    
    if len(cycle_3)==0: #if no cycles of 3 print N/A message
        writer.writerow(["N/A"])

    cycle_2 = []
    writer.writerow(["Cycle 2"])

    for (i,j) in G.edges: #checks for cycles of 2
        G.edges[(i,j)]["visited"] = False

    for (i,j) in G.edges:
        if G.edges[(i,j)]["visited"] == True: continue
        if (j,i) in G.edges:
            cycle_2.append((i,j,i))
            G.edges[(j,i)]["visited"] = True
            writer.writerow([i,j])
    if len(cycle_2)==0:
        writer.writerow(["N/A"])

    
#these are just check messages
print(len(potential_cycle_3))
print(len(cycle_3))
print(len(cycle_2))




20450880
0
32336


In [6]:
# Let's find a maximum matching
import gurobipy as gp
from gurobipy import GRB

# Create model object
m = gp.Model()

# Create variable for each edge
x = m.addVars(G.edges, vtype=GRB.BINARY)


# Objective function: maximize number of edges
m.setObjective(gp.quicksum(x[e] for e in G.edges), GRB.MAXIMIZE)

# The number of incomming arcs to each vertex is at most one
m.addConstrs(gp.quicksum(x[(u,v)] for u in G.neighbors(v) if (u,v) in G.edges) <= 1 for v in G.nodes)

# The number of incomming arcs should be equal to the number of outgoing arcs
m.addConstrs(gp.quicksum(x[(u,v)] for u in G.neighbors(v) if (u,v) in G.edges) == gp.quicksum(x[(v,u)] for u in G.neighbors(v) if (v,u) in G.edges) for v in G.nodes)

# Solve
m.optimize()

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2024-12-11
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS with Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1442 rows, 172284 columns and 301628 nonzeros
Model fingerprint: 0x132e4fea
Variable types: 0 continuous, 172284 integer (172284 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 722 rows and 106892 columns
Presolve time: 0.40s
Presolved: 720 rows, 65392 columns, 130424 nonzeros
Variable types: 0 continuous, 65392 integer (65392 binary)

Root relaxation

In [7]:
print("Objective:",m.objVal) #this determines the maximum amount of edges that can be developed 

selected_edges = [e for e in G.edges if x[e].X > 0.5]

with open("matches.csv","w",newline='') as file:
    writer = csv.writer(file)
    writer
    for i in selected_edges:
        writer.writerow([i])

Objective: 344.0
